In [1]:
import os
import nltk
import time

In [2]:
working_dir = 'C:/Users/JWeinstein/Capstone-main/src/'

In [3]:
def jaccard_similarity(list_x, list_y):
    'get jaccard similartiy of two documents'
    
    set_x = set(list_x)
    set_y = set(list_y)
    intersection = set_x.intersection(set_y)
    union = set_x.union(set_y)
    
    return len(intersection) / len(union) if len(union) > 0 else 0

def shingling_jaccard_similarity(text_x, text_y, n):
    'calculate similarity score between two documents using the shingling approach'
    
    x_ngrams=list(nltk.ngrams(text_x.split(" "),n))      
    y_ngrams=list(nltk.ngrams(text_y.split(" "),n))    
    # call function jaccard_similarity 
    sim_score=jaccard_similarity(x_ngrams,y_ngrams)
    
    return sim_score

In [4]:
# code adapted from https://dida.do/blog/how-to-identify-duplicate-files-with-python

directory = f"{working_dir}Raw_TXT_Downloads"
files = sorted(os.listdir(directory), key=lambda x: int(x.split(')')[0]))

# initialize list containing the classes of documents with the same content
duplicates = []

# get start time
startTime = time.time()

# comparison of the documents - nested loop approach
# loop through all files in given path
for file_name in files:
    # open document and get the text
    f = os.path.join(directory, file_name)
    with open(f, 'r', encoding="utf8") as file:
        url = file.readline().strip()
        title = file.readline().strip()
        text = file.read()
    
    is_duplicate = False
    
    # loop through list duplicates, calculate shingling_jaccard_similarity and check if it is duplicate by comparing 
    # the shingling_jaccard_similarity against a threshold of 0.95
    for class_ in duplicates:
        # open first document in class_ in list duplicates
        f = os.path.join(directory, class_[0])
        with open(f, 'r', encoding="utf8") as file:
            url = file.readline().strip()
            title = file.readline().strip()
            class_text = file.read()     
        
        # call function shingling_jaccard_similarity, with trigrams (n=3)
        shing_jaccard = shingling_jaccard_similarity(text, class_text, 3)
#         print(shing_jaccard)
        
        if shing_jaccard > 0.95:
            is_duplicate = True
        else:
            is_duplicate = False

#         print(is_duplicate)
        if is_duplicate:
            class_.append(file_name)            
            break
    
    if not is_duplicate:
        duplicates.append([file_name])     

# end time
endTime = time.time() - startTime

# show results: documents being together in one (inner) list are duplicates
(duplicates)


[['1) Health.txt',
  '87) Physical_health.txt',
  '487) Human_health.txt',
  '533) Physical_Health.txt'],
 ['2) Mental_health.txt',
  '24) Behavioral_health.txt',
  '185) Emotional_health.txt',
  '267) Mental_health_in_Canada.txt'],
 ['3) Health_care_provider.txt', '323) Healthcare_provider.txt'],
 ['4) Healthy_community_design.txt'],
 ['5) Healthy_city.txt'],
 ['6) Healthy_environment.txt'],
 ['7) Health_care.txt',
  '21) Health_services.txt',
  '135) Healthcare.txt',
  '229) Tertiary_healthcare.txt',
  '542) Healthcare_delivery.txt'],
 ['8) Social_determinants_of_health.txt',
  '249) Social_determinant_of_health.txt',
  '353) Determinants_of_health.txt'],
 ['9) Health_science.txt',
  '84) Health_profession.txt',
  '129) List_of_health_sciences_topics.txt',
  '130) Outline_of_health_sciences.txt',
  '308) Health_professions.txt',
  '351) Health_sciences.txt',
  '361) Topic_outline_of_health_science.txt'],
 ['10) Health_care_system.txt',
  '27) Health_system.txt',
  '91) Health_Care_Sy

In [5]:
endTime # in seconds

236.61902976036072

In [6]:
# remove the duplicates from directory: only keep the first document within the inner lists 
for class_ in duplicates:
    for file in class_[1:]:
        os.remove(os.path.join(directory, file))